In [2]:
#!pip3 install yfinance
#!pip3 install requests
import yfinance as yf
import numpy as np
import pandas as pd
import requests

In [20]:
stocks = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_500')[2]
stocks_data = stocks[3]
stocks_data = stocks_data + '.NS'
stocks_data = stocks_data.to_list()

In [30]:
#download the data since 1st Jan 2020
df = yf.download(stocks_data, start = '2020-01-01')['Close']

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['DUMMYSANOF.NS', 'SYMBOL.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


In [34]:
#calculate daily percentage return
ret_df = df.pct_change()

#cummulate the returns on a monthly basis
mtl_ret = (ret_df +1).resample('M').prod()

#calculating last 12 month rolling return
mtl_12 = mtl_ret.rolling(12).apply(np.prod).dropna()

#filering mechanism to identify the top 10 stocks
top_ = mtl_12.loc['2020-12-31'].nlargest(10)

#extracting relevant date 
top_.name

C:\Users\raviu\AppData\Local\Temp\ipykernel_29376\3586646961.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_df = df.pct_change()
C:\Users\raviu\AppData\Local\Temp\ipykernel_29376\3586646961.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mtl_ret = (ret_df +1).resample('M').prod()


Timestamp('2020-12-31 00:00:00')

In [39]:
#performance of the portfolio for subsequent month
mtl_ret[top_.name:][1:2]

#filter only the top performers from this
relevant_ret = mtl_ret[top_.name:][1:2][top_.index]
relevant_ret.mean(axis=1)

Date
2021-01-31    1.120385
Freq: ME, dtype: float64

In [40]:
#function with date as parameter

def top_performers(date):
    all_ = mtl_12.loc[date]
    top = all_.nlargest(10)
    relevant_ret = mtl_ret[top.name:][1:2][top.index]
    return (relevant_ret).mean(axis=1).values[0]

In [42]:
mom_ret = []

#pasing all relevant dates through the loop. momentum profits are stored in mom_ret
for date in mtl_12.index[:-1]:
    mom_ret.append(top_performers(date))

In [45]:
pd.Series(mom_ret).prod()

9.897382560048753